# PRMT-2458 Transfers requested breakdown for a practice - Aug - Oct 2021

## Context

A practice has advised there seems to be a large discrepancy between Transfers received and actual Registrations 

August 2021 
- Dashboard: 77 
- Registrations: 128 
- Difference: 51

September 2021 
- Dashboard: 161 
- Registrations: 237 
- Difference: 76

October 2021 
- Dashboard: 201 
- Registrations: 374
- Difference: 173

Total difference: 300

## Scope

Look at this practices full data for transfers in for the last three months, and show a breakdown of all transfers

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import paths
from data.practice_metadata import read_asid_metadata

pd.set_option('display.max_columns', None)

In [ ]:
asid_lookup=read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v2/2021/10/organisationMetadata.json")

transfer_file_location = "s3://prm-gp2gp-transfer-data-preprod/v6/2021/"

transfer_files = [
    "10/2021-10-transfers.parquet", 
    "9/2021-9-transfers.parquet",
    "8/2021-8-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]

transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers['month']=transfers['date_requested'].dt.to_period('M')

# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    "SystmOne" : "TPP",
    None: "Unknown"
}

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [ ]:
# filter for the practice as the requester
practice_of_interest_bool = transfers["requesting_practice_ods_code"]=="G81038"
practice_of_interest_transfers = transfers[practice_of_interest_bool].copy()
practice_of_interest_transfers.groupby(by=["month", "status"]).agg({"conversation_id": "count"})

In [ ]:
# get counts of failed transfers per month for the practice
failed_transfer_bool = practice_of_interest_transfers["status"]=="Technical failure"
practice_of_interest_failed_transfers = practice_of_interest_transfers[failed_transfer_bool]
practice_of_interest_monthly_failed_transfer_counts = (
    practice_of_interest_failed_transfers
        .groupby(by=["month"]).agg({"conversation_id": "count"})
        .rename(columns={"conversation_id": "Failed transfers"})
    )
practice_of_interest_monthly_failed_transfer_counts

In [ ]:
# Read MI RR records for the practice of interest (Aug - Oct)
dates_fields = ["RegistrationTime", "RequestFailureTime", "RequestTime", "ExtractTime", "ExtractAckTime", "ExtractAckFailureTime"]
practice_registrations = pd.read_csv("./practice_registrations_aug_to_oct.csv", parse_dates=dates_fields).fillna("None")

In [ ]:
# Check how many registrations didn't trigger GP2GP (no conversation id)
practice_registrations["ConversationID"].value_counts()

In [ ]:
# Filter for registrations that did not trigger GP2GP
no_conv_id_bool = practice_registrations["ConversationID"] == "None"
practice_registrations_with_no_conv_id = practice_registrations[no_conv_id_bool].copy()
practice_registrations_with_no_conv_id

In [ ]:
# Count by month registrations that did not trigger GP2GP
practice_registrations_with_no_conv_id['month'] = practice_registrations_with_no_conv_id['RegistrationTime'].dt.to_period('M')
monthly_practice_registrations_with_no_conv_id = (
    practice_registrations_with_no_conv_id
        .groupby(by=["month"]).agg({"RegistrationTime": "count"})
        .rename(columns={"RegistrationTime": "Non-GP2GP registrations"})
    )
monthly_practice_registrations_with_no_conv_id

In [ ]:
# Check how many registrations have the same requester and sender ODS code
practice_registrations_with_no_conv_id["SenderODS"].value_counts()

In [ ]:
# Merge failed transfers and registrations that did not trigger GP2GP
registrations = monthly_practice_registrations_with_no_conv_id.merge(practice_of_interest_monthly_failed_transfer_counts, left_index=True, right_index=True)
registrations["Total from our data"] = registrations.sum(axis=1)
registrations["Total from practice data"] = [51, 76, 173]
registrations.loc['Total']= registrations.sum()
registrations